In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import json
import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import random
import numpy as np


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
label_mapping = ["flooded",
"non flooded",
"flooded,non flooded",
"Yes",
"No",
"0",
"1",
"2",
"3",
"4",
"5",
"6",
"7",
"8",
"9",
"10",
"11",
"12",
"13",
"14",
"15",
"16",
"17",
"18",
"19",
"20",
"21",
"22",
"23",
"24",
"25",
"26",
"27",
"28",
"29",
"30",
"31",
"32",
"33",
"34",
"35",
"36",
"37",
"38",
"39",
"40",
"41",
"42",
"43",
"44",
"45",
"46",
"47",
"48",
"49",
"50"]

In [ ]:
OLD_FEATURE_PATH = "/content/drive/MyDrive/floodnet_features/"
FEATURE_PATH = "/content/drive/MyDrive/floodnet_convnext_features/"

In [ ]:
len(os.listdir(os.path.join(FEATURE_PATH, "Images/Train_Image")))

1448

In [ ]:
all_images_features = dict()
for imagename in os.listdir(os.path.join(FEATURE_PATH, "Images/Train_Image")):
    all_images_features[imagename.replace(".pt", ".JPG")] = torch.load(os.path.join(os.path.join(FEATURE_PATH, "Images/Train_Image"), imagename))

all_questions_features = dict()
for imagename in os.listdir(os.path.join(OLD_FEATURE_PATH, "Questions")):
    all_questions_features[imagename.replace(".pt", "")] = torch.load(os.path.join(os.path.join(OLD_FEATURE_PATH, "Questions"), imagename))


In [ ]:
all_questions_features.keys()

dict_keys(['What is the overall condition of the given image?', 'How many non flooded buildings can be seen in this image?', 'How many buildings can be seen in the image?', 'How many buildings can be seen in this image?', 'Is the entire road non flooded?', 'What is the condition of the road in this image?', 'How many buildings are non flooded?', 'Is the entire road flooded?', 'How many buildings are in this image?', 'What is the condition of road?', 'How many buildings are non flooded in this image?', 'How many buildings are in the image?', 'How many buildings are flooded in this image?', 'How many buildings are flooded?', 'How many flooded buildings can be seen in this image?'])

In [ ]:
class VQADataset(Dataset):
    def __init__(self, qdict, label_mapping=label_mapping):
        self.qdict = qdict
        self.label_mapping = label_mapping
        self.reset_index()

    def reset_index(self):
        new_qdict = dict()
        for idx, value in enumerate(self.qdict.values()):
            new_qdict[idx] = value
        self.qdict = new_qdict

    def __len__(self):
        return len(self.qdict.keys())
    
    def __getitem__(self, idx):
        # "0":{"Image_ID":"10165.JPG","Question":"What is the overall condition of the given image?","Ground_Truth":"flooded","Question_Type":"Condition_Recognition"}
        row = self.qdict[idx]
        img_feat = all_images_features[row["Image_ID"]]
        q_feat = all_questions_features[row["Question"]]
        
        return img_feat, q_feat.squeeze(), self.label_mapping.index(str(row["Ground_Truth"]))

In [ ]:
# !ls /content/drive/MyDrive/floodnet_data/Images/Train_Image | wc -l

1448


In [ ]:
qdict = json.load(open("/content/drive/MyDrive/floodnet_data/Questions/Training Question.json", "r"))

def get_train_val_splits(jsondict):
    total = len(jsondict.keys())
    train, val = train_test_split(list(jsondict.keys()), test_size=0.2)

    print(len(train),len(val))

    train_dict = dict()
    val_dict = dict()
    for i in train:
        train_dict[str(i)] = jsondict[str(i)]
    
    for j in val:
        val_dict[str(j)] = jsondict[str(j)]
    
    return train_dict, val_dict

def get_train_val_splits_typewise(jsondict):
    simple_count = dict()
    complex_count = dict()
    yes_no = dict()
    entire_image = dict()
    road_condition = dict()

    counter = 0
    for example in jsondict.values():
        if example["Question_Type"] == "Yes_No":
            yes_no[str(counter)] = example
            counter += 1
        elif example["Question_Type"] == "Simple_Counting":
            simple_count[str(counter)] = example
            counter += 1
        elif example["Question_Type"] == "Complex_Counting":
            complex_count[str(counter)] = example
            counter += 1
        elif "road" in example["Question"]:
            road_condition[str(counter)] = example
            counter += 1
        elif "overall" in example["Question"]:
            entire_image[str(counter)] = example
            counter += 1

    train_dict = dict()
    val_dict = dict()

    for typedict in [road_condition,simple_count,entire_image,complex_count,yes_no]:
        
        print(len(typedict.keys()))
        train, val = train_test_split(list(typedict.keys()), test_size=0.2)
        for idx, i in enumerate(train):
            
            if idx == 0:
                print(typedict[str(i)])
            train_dict[str(i)] = typedict[str(i)]
        
        for idx, j in enumerate(val):
            
            if idx == 0:
                print(typedict[str(j)])
            val_dict[str(j)] = typedict[str(j)]
        


    return train_dict, val_dict

def get_uniq_image_ids(jsondict):
    uniq_images = []
    for key, example in jsondict.items():
        if example["Image_ID"] not in uniq_images:
            uniq_images.append(example["Image_ID"])
    return uniq_images

def get_questions_for_imageid(jsondict, imageid):
    qs = dict()
    for key, example in jsondict.items():
        if example["Image_ID"] == imageid:
            qs[key] = example
    return qs

def get_train_val_splits_imagewise(jsondict):
    train_dict = dict()
    val_dict = dict()

    uniq_images = get_uniq_image_ids(jsondict)
    
    train, val = train_test_split(uniq_images, test_size=0.2)

    for imageid in train:
        train_dict.update(get_questions_for_imageid(jsondict, imageid)) 

    for imageid in val:
        val_dict.update(get_questions_for_imageid(jsondict, imageid))

    return train_dict, val_dict
train_dict, val_dict,  = get_train_val_splits_imagewise(qdict)

In [ ]:
print(len(train_dict.keys()))
print(len(val_dict.keys()))

3620
891


In [ ]:
train_ims = []
val_ims = []

for i in train_dict.values():
    if i["Image_ID"] not in train_ims:
        train_ims.append(i["Image_ID"])


for i in val_dict.values():
    if i["Image_ID"] not in val_ims:
        val_ims.append(i["Image_ID"])

In [ ]:
len(train_ims)

1158

In [ ]:
len(val_ims)

290

In [ ]:
def accuracy(pred, true):
    acc = np.sum((true == pred.argmax(-1)).astype(np.float32))
    return float(100 * acc / len(true))


In [ ]:
!mkdir ckpts

In [ ]:
# os.listdir(D)

NameError: ignored

In [ ]:
from datetime import datetime

class VQAModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linstack = nn.Sequential(
            # nn.Linear(in_features=3072, out_features=1024), # 3072 = 1024 (text) + 2048 (image)
            nn.Linear(in_features=2560, out_features=1024), # 3072 = 1024 (text) + 1536 (image)
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1024, out_features=512),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=256),
            nn.Linear(in_features=256, out_features=128),
            nn.Linear(in_features=128, out_features=56),
        )
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    def forward(self, batch):
        x = torch.concat([batch[0], batch[1]], dim=-1)
        x = self.linstack(x)
        return x

    def fit(self, train_ds, val_ds):
        now = datetime.now()
        timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
        os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
        self.to(self.device)
        train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
        val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4)

        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        loss_fn = nn.CrossEntropyLoss()

        for epoch in range(100):
            print(f"########## Epoch {epoch}")
            self.train()
            optimizer.zero_grad()
            
            epoch_loss = []
            epoch_acc = []
            
            val_loss = []
            val_acc = []

            for batch in tqdm.tqdm(train_dl):
                batch = [elem.to(self.device) for elem in batch]
        #         batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
                epoch_loss.append(loss.detach().cpu().numpy())
                epoch_acc.append(acc)
            
            self.eval()
            with torch.no_grad():
                for batch in tqdm.tqdm(val_dl):
                    batch = [elem.to(self.device) for elem in batch]
        #             batch[0] = tr(batch[0])
                    outputs = self(batch)
                    loss = loss_fn(outputs, batch[2])
            
                    acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                    val_loss.append(loss.detach().cpu().numpy())
                    val_acc.append(acc)
            
            
            torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")
            
            print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
            print(f"Val loss: {np.mean(val_loss)}", end="\t")
            print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
            print(f"Val acc: {np.mean(val_acc)}")
    
    def evaluate(self, val_ds):
        val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
        loss_fn = nn.CrossEntropyLoss()
        self.eval()
        self.to(self.device)
        val_loss = []
        val_acc = []
        with torch.no_grad():
            for batch in tqdm.tqdm(val_dl):
                batch = [elem.to(self.device) for elem in batch]
    #             batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
        
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                val_loss.append(loss.detach().cpu().numpy())
                val_acc.append(acc)
        print(f"Val loss: {np.mean(val_loss)}", end="\t")
        print(f"Val acc: {np.mean(val_acc)}")

In [ ]:
from datetime import datetime

class VQAModelProduct(nn.Module):
    def __init__(self):
        super().__init__()
        self.image = nn.Sequential(
         nn.Linear(in_features=1536, out_features=1024),
         nn.Dropout(p=0.2),
         nn.Linear(in_features=1024, out_features=512)   
        )
        self.txt = nn.Linear(in_features=1024, out_features=512)
        self.linstack = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=128),
            nn.Linear(in_features=128, out_features=56),
        )
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    def forward(self, batch):
        img = self.image(batch[0])
        txt = self.txt(batch[1])
        vec = img*txt
        # vec = torch.dot(img, txt) # check if they are 1-d tensors
        x = self.linstack(vec)
        return x

    def fit(self, train_ds, val_ds):
        now = datetime.now()
        timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
        os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
        self.to(self.device)
        train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
        val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4)

        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        loss_fn = nn.CrossEntropyLoss()

        for epoch in range(100):
            print(f"########## Epoch {epoch}")
            self.train()
            optimizer.zero_grad()
            
            epoch_loss = []
            epoch_acc = []
            
            val_loss = []
            val_acc = []

            for batch in tqdm.tqdm(train_dl):
                batch = [elem.to(self.device) for elem in batch]
        #         batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
                epoch_loss.append(loss.detach().cpu().numpy())
                epoch_acc.append(acc)
            
            self.eval()
            with torch.no_grad():
                for batch in tqdm.tqdm(val_dl):
                    batch = [elem.to(self.device) for elem in batch]
        #             batch[0] = tr(batch[0])
                    outputs = self(batch)
                    loss = loss_fn(outputs, batch[2])
            
                    acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                    val_loss.append(loss.detach().cpu().numpy())
                    val_acc.append(acc)
            
            
            torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")
            
            print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
            print(f"Val loss: {np.mean(val_loss)}", end="\t")
            print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
            print(f"Val acc: {np.mean(val_acc)}")
    
    def evaluate(self, val_ds):
        val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
        loss_fn = nn.CrossEntropyLoss()
        self.eval()
        self.to(self.device)
        val_loss = []
        val_acc = []
        with torch.no_grad():
            for batch in tqdm.tqdm(val_dl):
                batch = [elem.to(self.device) for elem in batch]
    #             batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
        
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                val_loss.append(loss.detach().cpu().numpy())
                val_acc.append(acc)
        print(f"Val loss: {np.mean(val_loss)}", end="\t")
        print(f"Val acc: {np.mean(val_acc)}")

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
from datetime import datetime

class VQAModelAdd(nn.Module):
    def __init__(self):
        super().__init__()
        self.image = nn.Sequential(
         nn.Linear(in_features=1536, out_features=1024),
         nn.Dropout(p=0.2),
         nn.Linear(in_features=1024, out_features=512)   
        )
        self.txt = nn.Linear(in_features=1024, out_features=512)
        self.linstack = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=128),
            nn.Linear(in_features=128, out_features=56),
        )
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    def forward(self, batch):
        img = self.image(batch[0])
        txt = self.txt(batch[1])
        vec = img+txt
        # vec = torch.dot(img, txt) # check if they are 1-d tensors
        x = self.linstack(vec)
        return x

    def fit(self, train_ds, val_ds):
        now = datetime.now()
        timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
        os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
        self.to(self.device)
        train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
        val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4)

        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        loss_fn = nn.CrossEntropyLoss()

        for epoch in range(100):
            print(f"########## Epoch {epoch}")
            self.train()
            optimizer.zero_grad()
            
            epoch_loss = []
            epoch_acc = []
            
            val_loss = []
            val_acc = []

            for batch in tqdm.tqdm(train_dl):
                batch = [elem.to(self.device) for elem in batch]
        #         batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
                epoch_loss.append(loss.detach().cpu().numpy())
                epoch_acc.append(acc)
            
            self.eval()
            with torch.no_grad():
                for batch in tqdm.tqdm(val_dl):
                    batch = [elem.to(self.device) for elem in batch]
        #             batch[0] = tr(batch[0])
                    outputs = self(batch)
                    loss = loss_fn(outputs, batch[2])
            
                    acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                    val_loss.append(loss.detach().cpu().numpy())
                    val_acc.append(acc)
            
            
            torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")
            
            print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
            print(f"Val loss: {np.mean(val_loss)}", end="\t")
            print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
            print(f"Val acc: {np.mean(val_acc)}")
    
    def evaluate(self, val_ds):
        val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
        loss_fn = nn.CrossEntropyLoss()
        self.eval()
        self.to(self.device)
        val_loss = []
        val_acc = []
        with torch.no_grad():
            for batch in tqdm.tqdm(val_dl):
                batch = [elem.to(self.device) for elem in batch]
    #             batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
        
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                val_loss.append(loss.detach().cpu().numpy())
                val_acc.append(acc)
        print(f"Val loss: {np.mean(val_loss)}", end="\t")
        print(f"Val acc: {np.mean(val_acc)}")

In [ ]:
a = torch.randn(8, 1024)
b = torch.randn(8, 1024)
multihead_attn = nn.MultiheadAttention(embed_dim=1024, num_heads=4)
attn_output, attn_output_weights = multihead_attn(query=b, key=a, value=a)
attn_output.shape, attn_output_weights.shape

(torch.Size([8, 1024]), torch.Size([8, 8]))

In [ ]:
a = torch.randn(8, 512)
b = torch.randn(8, 512)

dk = a.shape[-1]
k, v = a, a
q = b

# import numpy as np

(torch.nn.Softmax()(k@q.T / np.sqrt(dk)) @ v).shape


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


torch.Size([8, 512])

In [ ]:
from datetime import datetime

class AttentionBlock(nn.Module):
  def __init__(self, embed_dim=512, num_heads=4):
    super().__init__()
    self.multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
    self.lnorm = nn.LayerNorm(embed_dim)
    self.feed_forward = nn.Linear(embed_dim, embed_dim)

  def forward(self, k, v, q):
    attn_output, attn_output_weights = self.multihead_attn(query=q, key=k, value=v)
    attn_output = self.lnorm(attn_output + q)
    attn_output = self.lnorm(self.feed_forward(attn_output) + attn_output)
    return attn_output


class VQAModelAttention(nn.Module):
    def __init__(self, embed_dim=512, num_heads=4, num_blocks=1):
        super().__init__()
        self.attn = nn.ModuleList([
                               *(AttentionBlock(embed_dim=512, num_heads=4) for _ in range(num_blocks))
                                ])

        self.image = nn.Sequential(
         nn.Linear(in_features=2048, out_features=1024),
         nn.Dropout(p=0.2),
         nn.Linear(in_features=1024, out_features=512)   
        )
        self.txt = nn.Linear(in_features=1024, out_features=512)
        
        self.linstack = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=128),
            nn.Linear(in_features=128, out_features=56),
        )
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    def forward(self, batch):
        img = self.image(batch[0])
        txt = self.txt(batch[1])
        
        attn_output = img
        for layer in self.attn:
          attn_output = layer(k=attn_output, v=attn_output, q=txt)
          
        x = self.linstack(attn_output)
        return x

    def fit(self, train_ds, val_ds):
        now = datetime.now()
        timestr = now.strftime("%d_%m_%Hh%Mm%Ss")
        os.makedirs(f"./ckpts/{timestr}", exist_ok=True)
        self.to(self.device)
        train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
        val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4)

        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        loss_fn = nn.CrossEntropyLoss()

        for epoch in range(100):
            print(f"########## Epoch {epoch}")
            self.train()
            optimizer.zero_grad()
            
            epoch_loss = []
            epoch_acc = []
            
            val_loss = []
            val_acc = []

            for batch in tqdm.tqdm(train_dl):
                batch = [elem.to(self.device) for elem in batch]
        #         batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())
                
                epoch_loss.append(loss.detach().cpu().numpy())
                epoch_acc.append(acc)
            
            self.eval()
            with torch.no_grad():
                for batch in tqdm.tqdm(val_dl):
                    batch = [elem.to(self.device) for elem in batch]
        #             batch[0] = tr(batch[0])
                    outputs = self(batch)
                    loss = loss_fn(outputs, batch[2])
            
                    acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                    val_loss.append(loss.detach().cpu().numpy())
                    val_acc.append(acc)
            
            
            torch.save(self.state_dict(), f"./ckpts/{timestr}/vqa_{epoch}.pt")
            
            print(f"Train loss: {np.mean(epoch_loss)}", end="\t")
            print(f"Val loss: {np.mean(val_loss)}", end="\t")
            print(f"Train acc: {np.mean(epoch_acc)}", end="\t")
            print(f"Val acc: {np.mean(val_acc)}")
    
    def evaluate(self, val_ds):
        val_dl = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=4)
        loss_fn = nn.CrossEntropyLoss()
        self.eval()
        self.to(self.device)
        val_loss = []
        val_acc = []
        with torch.no_grad():
            for batch in tqdm.tqdm(val_dl):
                batch = [elem.to(self.device) for elem in batch]
    #             batch[0] = tr(batch[0])
                outputs = self(batch)
                loss = loss_fn(outputs, batch[2])
        
                acc = accuracy(outputs.detach().cpu().numpy(), batch[2].detach().cpu().numpy())

                val_loss.append(loss.detach().cpu().numpy())
                val_acc.append(acc)
        print(f"Val loss: {np.mean(val_loss)}", end="\t")
        print(f"Val acc: {np.mean(val_acc)}")

In [ ]:
train_ds = VQADataset(train_dict)
val_ds = VQADataset(val_dict)
len(val_ds)

891

In [ ]:
# for i in train_ds: 
#     print(i[0].shape)
#     print(i[1].shape)
#     break

In [ ]:
afdafdf

NameError: ignored

In [ ]:
# model = VQAModel()
# model = VQAModelProduct()
model = VQAModelAdd()
# model = VQAModelAttention()
model.fit(train_ds, val_ds)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


########## Epoch 0


100%|██████████| 7/7 [00:00<00:00, 14.09it/s]


Train loss: 2.2593133449554443	Val loss: 1.7740498781204224	Train acc: 43.87571839080459	Val acc: 51.40189459930314
########## Epoch 1


100%|██████████| 7/7 [00:00<00:00, 19.46it/s]


...

Train loss: 0.10700061917304993	Val loss: 1.6660000085830688	Train acc: 95.98299808429118	Val acc: 71.7207462253194
########## Epoch 98


100%|██████████| 7/7 [00:00<00:00, 19.84it/s]


Train loss: 0.10178519040346146	Val loss: 1.6476603746414185	Train acc: 96.56369731800766	Val acc: 71.60913908246225
########## Epoch 99


100%|██████████| 7/7 [00:00<00:00, 19.09it/s]

Train loss: 0.10086941719055176	Val loss: 1.702687382698059	Train acc: 96.41702586206897	Val acc: 71.38592479674797


In [ ]:
a = torch.tensor([1, 1, 2, 1])
b = torch.tensor([11, 12, 3, 2])
a*b

In [ ]:
simple_count = dict()
complex_count = dict()
yes_no = dict()
entire_image = dict()
road_condition = dict()

counter = 0
for example in val_dict.values():
    if example["Question_Type"] == "Yes_No":
        yes_no[str(counter)] = example
        counter += 1
    elif example["Question_Type"] == "Simple_Counting":
        simple_count[str(counter)] = example
        counter += 1
    elif example["Question_Type"] == "Complex_Counting":
        complex_count[str(counter)] = example
        counter += 1
    elif "road" in example["Question"]:
        road_condition[str(counter)] = example
        counter += 1
    elif "overall" in example["Question"]:
        entire_image[str(counter)] = example
        counter += 1

simple_count = VQADataset(simple_count)
complex_count = VQADataset(complex_count)
road_condition = VQADataset(road_condition)
yes_no = VQADataset(yes_no)
entire_image = VQADataset(entire_image)

train_ds = VQADataset(train_dict)
val_ds = VQADataset(val_dict)
len(val_ds)

891

In [ ]:
print(len(simple_count))
print(len(complex_count))
print(len(yes_no))
print(len(entire_image))
print(len(road_condition))

116
131
177
290
177


In [ ]:
a = [116,
131,
177,
290,
177]
sum(a)

891

In [ ]:
!ls /content/ckpts/

29_05_19h34m20s  29_05_20h17m39s  29_05_20h22m14s
29_05_19h39m21s  29_05_20h18m10s  29_05_20h26m25s


In [ ]:
# model = VQAModelAttention()
# model = VQAModel()
model = VQAModelAdd()
# model = VQAModelProduct()
model.load_state_dict(torch.load("/content/ckpts/29_05_20h26m25s/vqa_87.pt"))
model.eval()

VQAModel(
  (linstack): Sequential(
    (0): Linear(in_features=2560, out_features=1024, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): Linear(in_features=128, out_features=56, bias=True)
  )
)

In [ ]:
model.evaluate(val_ds)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 891/891 [00:03<00:00, 290.10it/s]

Val loss: 1.653978705406189	Val acc: 74.41077441077441


In [ ]:
model.evaluate(simple_count)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 116/116 [00:00<00:00, 180.36it/s]

Val loss: 4.336513996124268	Val acc: 40.51724137931034


In [ ]:
model.evaluate(complex_count)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 131/131 [00:00<00:00, 193.42it/s]

Val loss: 4.718360424041748	Val acc: 35.11450381679389


In [ ]:
model.evaluate(yes_no)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 177/177 [00:00<00:00, 219.75it/s]

Val loss: 1.7234106063842773	Val acc: 63.84180790960452


In [ ]:
model.evaluate(entire_image)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 290/290 [00:01<00:00, 260.02it/s]

Val loss: 0.062075234949588776	Val acc: 98.27586206896552


In [ ]:
model.evaluate(road_condition)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 177/177 [00:00<00:00, 223.00it/s]

Val loss: 0.16671614348888397	Val acc: 97.17514124293785


In [ ]:
!pip install transformers
# from transformers import ViLT

In [ ]:
from transformers import ViltModel

In [ ]:
from transformers import ViltProcessor, ViltModel
from PIL import Image
import requests

# prepare image and text
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "hello world"

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
model = ViltModel.from_pretrained("dandelin/vilt-b32-mlm")

inputs = processor(image, text, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)